In [1]:
%%capture 
%run BaseDélinquance.ipynb

In [6]:
%%capture 
%run BaseEclairage_v2.ipynb

In [13]:
%%capture 
%run Basevideoprotection.ipynb

In [21]:
%%capture 
%run BasePauverete.ipynb

In [19]:
%run fonctions.py
delinquance_par_arrondissement['arrondissement'] = delinquance_par_arrondissement['arrondissement'].apply(extraire_arrondissement)

In [42]:
delinquance_par_arrondissement.rename(columns={"arrondissement": "INSEE_COG"}, inplace=True)


In [36]:
df_log_flt = df_log[['INSEE_COG', 'nb_log']]
df_log_flt = df_log_flt.drop_duplicates()
df_log_flt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 196
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   INSEE_COG  20 non-null     object
 1   nb_log     20 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 480.0+ bytes


In [32]:
delinquance_par_arrondissement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   INSEE_COG      20 non-null     object 
 1   tauxpourmille  20 non-null     float64
dtypes: float64(1), object(1)
memory usage: 448.0+ bytes


In [43]:
df_couverture_videoprotection_par_arrondissement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   INSEE_COG       20 non-null     object  
 1   nombre_cameras  20 non-null     int64   
 2   geometry        20 non-null     geometry
dtypes: geometry(1), int64(1), object(1)
memory usage: 608.0+ bytes


In [25]:
df_pauv_flt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 3321 to 3340
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   INSEE_COG  20 non-null     object 
 1   LIBGEO     20 non-null     object 
 2   TP4017     20 non-null     float64
 3   TP5017     20 non-null     float64
 4   TP6017     20 non-null     float64
 5   TP60Q217   20 non-null     int64  
 6   TP60IP17   20 non-null     float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.2+ KB


In [26]:
df_commerces.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 58 to 1273
Data columns (total 34 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   departement                                     20 non-null     int64 
 1   INSEE_COG                                       20 non-null     int64 
 2   libelle_de_commune                              20 non-null     object
 3   arrondissement                                  20 non-null     int64 
 4   canton_ville                                    20 non-null     int64 
 5   zone_d_emploi                                   20 non-null     int64 
 6   unite_urbaine                                   20 non-null     int64 
 7   population_2010                                 20 non-null     int64 
 8   hypermarche                                     20 non-null     int64 
 9   supermarche                                     20 non-nul

In [27]:
df_densite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   c_ar                    20 non-null     int64  
 1   INSEE_COG               20 non-null     int64  
 2   l_ar                    20 non-null     object 
 3   surface                 20 non-null     float64
 4   geom                    20 non-null     object 
 5   Nombre_points_lumineux  20 non-null     int64  
 6   densité_lum             20 non-null     float64
 7   geometry                20 non-null     object 
dtypes: float64(2), int64(3), object(3)
memory usage: 1.4+ KB


In [39]:
import pandas as pd

df_final = df_densite[['INSEE_COG', 'densité_lum']].copy()
dataframes = [
    df_couverture_videoprotection_par_arrondissement[['INSEE_COG', 'nombre']].copy(),
    delinquance_par_arrondissement.copy(),
    df_pauv_flt[['INSEE_COG', 'TP6017']].copy(),
    df_commerces[['INSEE_COG', 'total_commerces']].copy(),
    df_log_flt.copy()
]
#Unifier le type de INSEE_COG car dans certains df il est de type int64 et dans d'autres de type object
for df in dataframes:
    df['INSEE_COG'] = df['INSEE_COG'].astype(str)
    print(f"Types after conversion: {df['INSEE_COG'].dtype}")

df_final['INSEE_COG'] = df_final['INSEE_COG'].astype(str)
print(f"Final DF type for INSEE_COG: {df_final['INSEE_COG'].dtype}")
for df in dataframes:
    df_final = pd.merge(df_final, df, on='INSEE_COG', how='inner')

df_final.head()



Types after conversion: object
Types after conversion: object
Types after conversion: object
Types after conversion: object
Types after conversion: object
Final DF type for INSEE_COG: object


,INSEE_COG,densité_lum,nombre,tauxpourmille,TP6017,total_commerces,nb_log
0,75105,1345.606894,30,912.244964,10.8,754,81
1,75110,1217.606244,43,1632.879669,17.4,1395,218
2,75118,1395.251570,57,1095.573722,21.2,1713,512
3,75107,1272.598344,45,980.394686,7.9,899,26
4,75106,1088.200642,23,1170.924386,9.3,1393,48


In [ ]:
##Après avoir tout rééxecuter renommer les colonnes directement dans df_final et appliquer ACP et Regression